In [8]:
import os
import glob
import random
import pathlib
import numpy as np
import tensorflow as tf
import IPython.display as display
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter

%matplotlib inline

In [9]:
def test_device():
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
    gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    print('GPU Info')
    for gpu in gpus:
        print('   ', gpu)
    print('CPU Info')
    for cpu in cpus:
        print('   ', cpu)
    
    print()
    if gpus:
        try:
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
            tf.config.experimental.set_memory_growth(gpus[0], True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        except RuntimeError as e:
            print(e)
        
test_device()
tf.keras.backend.clear_session() 
AUTOTUNE = tf.data.experimental.AUTOTUNE

GPU Info
    PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
CPU Info
    PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')

1 Physical GPUs, 1 Logical GPU


In [10]:
def change_range(image):
    return 2*image-1

def preprocess_image(image, img_size):
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [img_size, img_size])
    image /= 255.0 
    return change_range(image)

def load_and_preprocess_image(path, img_size):
    image = tf.io.read_file(path)
    return preprocess_image(image, img_size)

def load_and_preprocess_from_path(path, img_size):
    return load_and_preprocess_image(path, img_size)

In [11]:
test_data_root_orig = '/root/aoi/test_images'
test_csv = '/root/aoi/test.csv'
model_path = './model/checkpoints_128_32_888_0/'
test_image_paths = glob.glob('/'.join([test_data_root_orig, '*.png']))
test_image_paths.sort()
img_size = 128

In [12]:
model = tf.saved_model.load(model_path)

In [14]:
preds = []
for idx in tqdm(range(len(test_image_paths))):
    path = test_image_paths[idx]
    name = path.split('/')[-1]
    ID = int(name.split('.png')[0].replace('test_', ''))
    img = load_and_preprocess_from_path(path, img_size)
    img = np.array(img).reshape(-1, img_size, img_size, 1)
    preds.append(model(img).numpy())

100%|██████████| 10142/10142 [01:30<00:00, 112.24it/s]


In [15]:
if os.path.exists(test_csv):
    os.remove(test_csv)
with open(test_csv, 'a') as csv:
    csv.write('ID,Label')
    for ID, pred in enumerate(preds):
        csv.write('\ntest_{:05}.png,{}'.format(ID, np.argmax(pred)))